In [1]:
# import psycopg2
from sqlalchemy import create_engine
import pandas as pd

# Create a class to create pandas dataframe from SQL queries

In [142]:
#Read data from JDBC server
class DB:
    def __init__(self):
        self.tables = None
        self.df = None
        self.point = None
        self.POSTGRES_HOST = "codd03.research.northwestern.edu"
        self.POSTGRES_PORT = 5432
        self.POSTGRES_DBNAME = 'cpdb'
        self.POSTGRES_USERNAME = 'cpdb_student'
        self.POSTGRES_PASSWORD = 'DataSci4Lyfe'
        self.postgres_str = ('postgresql://{username}:{password}@{host}:{port}/{dbname}'.format(username=self.POSTGRES_USERNAME,
                                                                                           password=self.POSTGRES_PASSWORD,
                                                                                           host=self.POSTGRES_HOST,
                                                                                           port=self.POSTGRES_PORT,
                                                                                           dbname=self.POSTGRES_DBNAME))
        self.engine = create_engine(self.postgres_str)

    def query(self, queryCommand):
            self.point = self.engine.execute(queryCommand)
            self.df = None
            try: self.df = pd.DataFrame([*self.point.mappings()])
            except Exception as msg:
                if msg == 'This result object does not return rows. It has been closed automatically.':pass
            return self.df

    def get_tables_from_commands(self, commands=None):
        self.tables = {}
        for command in commands:
            try:
                df = self.query(command)
                if df is not None:
                    self.tables[f'table_{len(self.tables.keys())}'] = {'frame': df, 'sql': command}
            except:
                pass
        return self.tables



In [148]:
# test it out

def test():
    obj_test = DB()
    t = obj_test.get_tables_from_commands

    query1 = '''
    DROP TABLE if exists area_ids;
    CREATE TEMP TABLE area_ids AS
        ( SELECT id, name FROM data_area
          WHERE area_type='police-districts');
    SELECT * FROM area_ids;

    -- Determining civilian population of each PD
    DROP TABLE IF EXISTS popul_by_pd;
    CREATE TEMP TABLE popul_by_pd AS
        ( SELECT area_id, SUM("count") population FROM data_racepopulation
          WHERE area_id IN (SELECT id FROM area_ids)
          group by area_id);
    select * from popul_by_pd;
    '''

    commands = query1.split(';')
    tables = t(commands)
    if len(tables.keys()) == 2: print("Passed!")
    else: print("An error occurred")

    del obj_test

test()

Passed!


# Get all required tables

In [145]:
filename = 'C:\\Users\\KIIT\\AppData\\Roaming\\JetBrains\\DataSpell2022.2\\consoles\\db\\04e55ffa-c834-4191-a73f-da9162a9172f\\console.sql'

with open(filename, 'r') as f:
    sqlFile = f.read()
commands_raw = sqlFile.split(';')
commands = []
for command in commands_raw:
    q = f'''
    {command}
    '''
    commands.append(q)
tables = t(commands)
for key in tables.keys(): print(key)


table_0
table_1
table_2
table_3
table_4
table_5
table_6
table_7
table_8
table_9
